In [1]:
!pip install contractions
import warnings
warnings.filterwarnings('ignore')
import statistics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import load_digits
from sklearn.linear_model import Perceptron
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
! pip install bs4  # in case you don't have it installed

     |████████████████████████████████| 296kB 8.2MB/s eta 0:00:01


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/weizicai/nltk_data...


  Stored in directory: /Users/weizicai/Library/Caches/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [22]:
df = pd.read_csv('train.csv', sep=',',error_bad_lines=False)
df

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5
5,004AC288D833,"Dear Principal,\r\n\r\nOur school should have ...",3.5,4.0,4.0,3.5,3.5,4.0
6,5661280443,Imagine if you could prove other people that y...,3.5,4.0,3.5,3.5,4.0,4.0
7,008DDDDD8E8D,I think it's a good idea for the estudnets to ...,2.5,2.5,2.5,2.5,2.5,2.0
8,009BCCC61C2A,positive attitude is the key to success. I agr...,3.0,3.0,3.5,3.5,3.0,3.0
9,009F4E9310CB,Asking more than one person for and advice hel...,3.0,3.0,3.5,2.5,3.0,2.5


In [26]:
df['cohesion'] = pd.to_numeric(df['cohesion'], errors='coerce').astype(str)
df['syntax'] = pd.to_numeric(df['syntax'], errors='coerce').astype(str)
df['vocabulary'] = pd.to_numeric(df['vocabulary'], errors='coerce').astype(str)
df['phraseology'] = pd.to_numeric(df['phraseology'], errors='coerce').astype(str)
df['grammar'] = pd.to_numeric(df['grammar'], errors='coerce').astype(str)
df['conventions'] = pd.to_numeric(df['conventions'], errors='coerce').astype(str)
rev_and_rating = df.rename({'full_text': 'review_body'}, axis=1)

In [28]:
rev_and_rating

,text_id,review_body,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5
5,004AC288D833,"Dear Principal,\r\n\r\nOur school should have ...",3.5,4.0,4.0,3.5,3.5,4.0
6,5661280443,Imagine if you could prove other people that y...,3.5,4.0,3.5,3.5,4.0,4.0
7,008DDDDD8E8D,I think it's a good idea for the estudnets to ...,2.5,2.5,2.5,2.5,2.5,2.0
8,009BCCC61C2A,positive attitude is the key to success. I agr...,3.0,3.0,3.5,3.5,3.0,3.0
9,009F4E9310CB,Asking more than one person for and advice hel...,3.0,3.0,3.5,2.5,3.0,2.5


In [29]:
balanced_data = rev_and_rating
balanced_data['review_body'] = balanced_data['review_body'].str.lower()
import contractions
balanced_data['review_body']=balanced_data['review_body'].apply(lambda x: contractions.fix(str(x)))
import re
balanced_data['review_body'] = balanced_data['review_body'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
balanced_data['review_body'] = balanced_data['review_body'].apply(lambda x: re.sub('<[^<]+?>', '', str(x)))
balanced_data['review_body'] = balanced_data['review_body'].str.replace('[^a-zA-Z ]', '')
nltk.download('stopwords')
stop_words = stopwords.words('english')
balanced_data['review_body'] = balanced_data['review_body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('omw-1.4')
balanced_data['review_body'] = balanced_data['review_body'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
balanced_data['review_body'] = balanced_data['review_body'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word, pos ="a") for word in x.split()]))
balanced_data['review_body'] = balanced_data['review_body'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word, pos ="v") for word in x.split()]))
balanced_data['review_body'] = balanced_data['review_body'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word, pos ="r") for word in x.split()]))
balanced_data['review_body'] = balanced_data['review_body'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word, pos ="s") for word in x.split()]))
balanced_data = balanced_data.reset_index()
del balanced_data['index']


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/weizicai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/weizicai/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [40]:
X = balanced_data['review_body']
y = balanced_data['cohesion']
tfIdfVectorizer1=TfidfVectorizer(max_features=1000)
tfIdf_x = tfIdfVectorizer1.fit_transform(X)
tfIdf_x_array = tfIdf_x.toarray()
#feature_names = tfIdfVectorizer1.get_feature_names()
#tfIdf_x_df = pd.DataFrame(tfIdf_x_array, columns = feature_names)

In [74]:
train_x, test_x, train_y, test_y = train_test_split(tfIdf_x_array, y, test_size=0.2, random_state=42)

# Applying SMOTE on train dataset

In [159]:
from collections import Counter
Counter(train_y)

Counter({'3.5': 723,
         '3.0': 920,
         '2.5': 628,
         '2.0': 321,
         '1.5': 17,
         '4.5': 95,
         '4.0': 395,
         '5.0': 19,
         '1.0': 10})

In [160]:
from imblearn.over_sampling import SMOTE
smote=SMOTE('not minority')
train_x,train_y=smote.fit_resample(train_x,train_y)

In [161]:
Counter(train_y)

Counter({'3.5': 920,
         '3.0': 920,
         '2.5': 920,
         '2.0': 920,
         '1.5': 920,
         '4.5': 920,
         '4.0': 920,
         '5.0': 920,
         '1.0': 10})

# SVM with grid search CV

In [105]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [141]:
## cohesion
y = balanced_data['cohesion']
train_x, test_x, train_y, test_y = train_test_split(tfIdf_x_array, y, test_size=0.2, random_state=42)
nb_clf = MultinomialNB()
nb_clf.fit(train_x, train_y)
nb_pred_y = nb_clf.predict(test_x)
print('without cross validation: ',accuracy_score(test_y, nb_pred_y))
parameters = {'alpha':(1, 0.1, 0.01, 0.001, 0.0001, 0.00001)}

nb_clf_cv = GridSearchCV(MultinomialNB(), parameters)
nb_clf_cv.fit(train_x, train_y)
nb_cv_pred_y = nb_clf_cv.predict(test_x)

print('with cross validation: ',accuracy_score(test_y, nb_cv_pred_y))

without cross validation:  0.29246487867177523
with cross validation:  0.29246487867177523


In [155]:
## syntax
y = balanced_data['syntax']
train_x, test_x, train_y, test_y = train_test_split(tfIdf_x_array, y, test_size=0.2, random_state=42)
nb_clf = MultinomialNB()
nb_clf.fit(train_x, train_y)
nb_pred_y = nb_clf.predict(test_x)
print('without cross validation: %.3f' % accuracy_score(test_y, nb_pred_y))
parameters = {'alpha':(1, 0.1, 0.01, 0.001, 0.0001, 0.00001)}

nb_clf_cv = GridSearchCV(MultinomialNB(), parameters)
nb_clf_cv.fit(train_x, train_y)
nb_cv_pred_y = nb_clf_cv.predict(test_x)

print('with cross validation: %.3f' % accuracy_score(test_y, nb_cv_pred_y))

without cross validation: 0.296
with cross validation: 0.310


In [153]:
## Vocabulary
y = balanced_data['vocabulary']
train_x, test_x, train_y, test_y = train_test_split(tfIdf_x_array, y, test_size=0.2, random_state=42)
nb_clf = MultinomialNB()
nb_clf.fit(train_x, train_y)
nb_pred_y = nb_clf.predict(test_x)
print('without cross validation: %.3f' % accuracy_score(test_y, nb_pred_y))
parameters = {'alpha':(1, 0.1, 0.01, 0.001, 0.0001, 0.00001)}

nb_clf_cv = GridSearchCV(MultinomialNB(), parameters)
nb_clf_cv.fit(train_x, train_y)
nb_cv_pred_y = nb_clf_cv.predict(test_x)

print('with cross validation: %.3f' % accuracy_score(test_y, nb_cv_pred_y))

without cross validation: 0.379
with cross validation: 0.401


In [156]:
## phraseology
y = balanced_data['phraseology']
train_x, test_x, train_y, test_y = train_test_split(tfIdf_x_array, y, test_size=0.2, random_state=42)
nb_clf = MultinomialNB()
nb_clf.fit(train_x, train_y)
nb_pred_y = nb_clf.predict(test_x)
print('without cross validation: %.3f' % accuracy_score(test_y, nb_pred_y))
parameters = {'alpha':(1, 0.1, 0.01, 0.001, 0.0001, 0.00001)}

nb_clf_cv = GridSearchCV(MultinomialNB(), parameters)
nb_clf_cv.fit(train_x, train_y)
nb_cv_pred_y = nb_clf_cv.predict(test_x)

print('with cross validation: %.3f' % accuracy_score(test_y, nb_cv_pred_y))

without cross validation: 0.285
with cross validation: 0.313


In [157]:
## grammar
y = balanced_data['grammar']
train_x, test_x, train_y, test_y = train_test_split(tfIdf_x_array, y, test_size=0.2, random_state=42)
nb_clf = MultinomialNB()
nb_clf.fit(train_x, train_y)
nb_pred_y = nb_clf.predict(test_x)
print('without cross validation: %.3f' % accuracy_score(test_y, nb_pred_y))
parameters = {'alpha':(1, 0.1, 0.01, 0.001, 0.0001, 0.00001)}

nb_clf_cv = GridSearchCV(MultinomialNB(), parameters)
nb_clf_cv.fit(train_x, train_y)
nb_cv_pred_y = nb_clf_cv.predict(test_x)

print('with cross validation: %.3f' % accuracy_score(test_y, nb_cv_pred_y))

without cross validation: 0.290
with cross validation: 0.311


In [158]:
## conventions
y = balanced_data['conventions']
train_x, test_x, train_y, test_y = train_test_split(tfIdf_x_array, y, test_size=0.2, random_state=42)
nb_clf = MultinomialNB()
nb_clf.fit(train_x, train_y)
nb_pred_y = nb_clf.predict(test_x)
print('without cross validation: %.3f' % accuracy_score(test_y, nb_pred_y))
parameters = {'alpha':(1, 0.1, 0.01, 0.001, 0.0001, 0.00001)}

nb_clf_cv = GridSearchCV(MultinomialNB(), parameters)
nb_clf_cv.fit(train_x, train_y)
nb_cv_pred_y = nb_clf_cv.predict(test_x)

print('with cross validation: %.3f' % accuracy_score(test_y, nb_cv_pred_y))

without cross validation: 0.290
with cross validation: 0.320
